In [1]:
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from collections import Counter
nltk.download('punkt')
import pandas as pd
import re

[nltk_data] Downloading package punkt to C:\Users\niraj
[nltk_data]     mahi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:

nltk.data.path.append('.')

In [7]:

file1 = r'final_review_dataset.csv'


df1 = pd.read_csv(file1, on_bad_lines='skip', low_memory=False)


From now the  modification of context,center and sentiment label is done

In [83]:
def get_windows(words, C):
    
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[(i - C):i] + words[(i + 1):(i + C + 1)]
        yield context_words, center_word
        i += 1






C=1


In [84]:
def to_one_hot(data_point_index, vocabsize):
    temp = np.zeros(vocabsize)
    temp[data_point_index] = 1
    return temp

In [86]:


file1 = r'final_review_dataset.csv'



df = pd.read_csv(file1, on_bad_lines='skip', low_memory=False)




In [87]:
df1 = df.sample(n=100, random_state=42)

In [88]:
df1.reset_index(drop=True, inplace=True)


In [89]:
df1['reviewText']

0     box says works   feet away   nt tested yetbut ...
1                                 product registeration
2     overall wierd movie   saw description guide sa...
3                                            piece junk
4     mark hamon character twist   turns worth purchase
                            ...                        
95                                           ugly weird
96                 router good wireless card nt connect
97                                        great cameras
98    finally clint eastwood movie belong dirty harr...
99                                best car receiver far
Name: reviewText, Length: 100, dtype: object

In [90]:
wordList = []  
vocabulary = set()  

for review_text in df1['reviewText']:
    words = review_text.split()  
    wordList.extend(words)  # Add all words to wordList
    vocabulary.update(words)  

print("Total number of words in wordList:", len(wordList))
print("Total number of unique words in vocabulary:", len(vocabulary))

Total number of words in wordList: 3249
Total number of unique words in vocabulary: 1952


In [91]:
vocabsize= len(vocabulary)

In [92]:
word_2_int={}
int_2_word={}

In [93]:
wordList = list(vocabulary)  # Assuming vocabulary contains the unique words
print("Number of words in wordList:", len(wordList))
word_2_int={}
int_2_word={}
for i, word in enumerate(wordList):
    word_2_int[word] = i  
    int_2_word[i] = word

print("Number of words in word2int:", len(word_2_int))
print("Number of indices in int2word:", len(int_2_word))

Number of words in wordList: 1952
Number of words in word2int: 1952
Number of indices in int2word: 1952


In [94]:
context_data = []
senti_data = []
center_data=[]
i = 0
for index, row in df1.iterrows():
    
  
    words = row['reviewText'].split()
    sentiment_label = row['review_label']
   

    for context_words, center_word  in get_windows(words, C):
        context_data.append(context_words)
        senti_data.append(sentiment_label)
        center_data.append(center_word)
    i=i+1
#print(context_data)
#print(senti_data)
print(len(context_data))
print(len(senti_data))
print(i)
print(context_data[0])


3052
3052
100
['box', 'works']


### Context Target Word  one hot encoded

In [95]:
context_data_hot = []

center_data_hot=[]

i=0

for context_word,center_word in zip(context_data,center_data):
    
    
    
    context_words_vectors = [to_one_hot(word_2_int[w], vocabsize) for w in context_word]
    context_data_hot.append(np.mean(context_words_vectors,axis=0))
    center_data_hot.append(to_one_hot(word_2_int[center_word], vocabsize))
    
    i=i+1
print(len( context_words_vectors ))
print(context_data_hot[0])

2
[0. 0. 0. ... 0. 0. 0.]


## Batch Creation

In [96]:
def get_batches(batch_size):
    batch_x = []
    batch_y = []
    batch_senti=[]
    for x, y,z in zip ( context_data_hot,center_data_hot,senti_data):
        while len(batch_x) < batch_size:
            batch_x.append(x)
            batch_y.append(y)
            batch_senti.append(z)
        else:
            yield np.array(batch_x).T, np.array(batch_y).T,np.array(batch_senti).T
            batch = []

In [97]:
i=0

for x,y, z in get_batches(7):
    print(x.shape)
    print(y.shape)
    print(z.shape)
    i=i+1
    if i==1:
        break

(1952, 7)
(1952, 7)
(7,)


In [8]:
df1['reviewText'] = df1['reviewText'].apply(lambda x: re.sub(r'[,!?;-]', '.', x))

# Tokenize and preprocess the data
data = nltk.word_tokenize(df1['reviewText'].str.cat(sep=' '))
data = [ch.lower() for ch in data if ch.isalpha() or ch == '.']

In [33]:
import numpy as np
from scipy import linalg
from collections import defaultdict


def sigmoid(z):
   
    return 1.0/(1.0+np.exp(-z))


def get_idx(words, word2Ind):
    idx = []
    for word in words:
        idx = idx + [word2Ind[word]]
    return idx


def pack_idx_with_frequency(context_words, word2Ind):
    freq_dict = defaultdict(int)
    for word in context_words:
        freq_dict[word] += 1
    idxs = get_idx(context_words, word2Ind)
    packed = []
    for i in range(len(idxs)):
        idx = idxs[i]
        freq = freq_dict[context_words[i]]
        packed.append((idx, freq))
    print("packed")
    print(packed)
    return packed


def get_vectors(data, word2Ind, V, C):
    i = C
    while True:
        y = np.zeros(V)
        x = np.zeros(V)
        center_word = data[i]
        y[word2Ind[center_word]] = 1
        context_words = data[(i - C):i] + data[(i+1):(i+C+1)]
        print("context_word")
        print(context_words)
        num_ctx_words = len(context_words)
        for idx, freq in pack_idx_with_frequency(context_words, word2Ind):
            x[idx] = freq/num_ctx_words
        yield x, y
        i += 1
        if i >= len(data):
            print('i is being set to 0')
            i = 0





def compute_pca(data, n_components=2):
   
    m, n = data.shape

   
    data -= data.mean(axis=0)
  
    R = np.cov(data, rowvar=False)
   
    evals, evecs = linalg.eigh(R)
   
    idx = np.argsort(evals)[::-1]

    evecs = evecs[:, idx]
   
    evals = evals[idx]
 
    evecs = evecs[:, :n_components]
   
    return np.dot(evecs.T, data.T).T


def get_dict(data):
  
    words = sorted(list(set(data)))
    n = len(words)
    idx = 0
  
    word2Ind = {}
    Ind2word = {}
    for k in words:
        word2Ind[k] = idx
        Ind2word[idx] = k
        idx += 1
    return word2Ind, Ind2word

## From there  this will modified.

In [153]:
def initialize_model(N,V,sentiment_classes, random_seed=1):
   
    np.random.seed(random_seed)
    
   
    
    W1 = np.random.rand(N,V)
   
    W2 = np.random.rand(V,N)
   
    b1 = np.random.rand(N,1)
   
    b2 = np.random.rand(V,1)

    # Sentiment model parameters (logistic regression)
    W_s = np.random.rand(1, N)  # Assuming binary classification for simplicity
    b_s = np.random.rand(1, 1)
   

    return W1, W2, b1, b2,W_s,b_s

In [140]:
def softmax(z):
   
    e_z = np.exp(z)
    yhat = e_z/np.sum(e_z,axis=0)
  
    
    return yhat

In [141]:
def forward_prop(x, W1, W2, b1, b2,W_sentiment,b_sentiment):
   
    
   
    h = np.dot(W1,x)+b1
    
   
    h = np.maximum(0,h)
    
   
    z = np.dot(W2,h)+b2

    

    
    

    return z, h 

In [142]:

#  cross-entropy cost functioN
def compute_cost(y, yhat, batch_size):
    # cost function 
    logprobs = np.multiply(np.log(yhat),y) + np.multiply(np.log(1 - yhat), 1 - y)
    cost = - 1/batch_size * np.sum(logprobs)
    cost = np.squeeze(cost)
    return cost


In [171]:
def compute_sentiment_cost(y_sentiment, pred_s, batch_size):
    # Binary cross-entropy cost for sentiment classification
   epsilon = 1e-7  # A small constant
   cost_s = -1 / batch_size * np.sum(y_sentiment * np.log(pred_s + epsilon) + (1 - y_sentiment) * np.log(1 - pred_s + epsilon))

   cost_s = np.squeeze(cost_s)
   return cost_s

In [177]:
def back_prop(x, yhat, y, h, W1, W2, b1, b2, W_s,b_s, pred_s,y_sentiment, batch_size):
    l1 = np.dot(W2.T, (yhat - y))
    #  relu to l1
    l1 = np.maximum(0, l1)
    #  the gradient of W1
    grad_W1 = np.dot(l1, x.T) / batch_size
    #  the gradient of W2
    grad_W2 = np.dot(yhat - y, h.T) / batch_size
    #  the gradient of b1
    grad_b1 = np.sum(l1, axis=1, keepdims=True) / batch_size
    # the gradient of b2
    grad_b2 = np.sum(yhat - y, axis=1, keepdims=True) / batch_size

    ds = pred_s - y_sentiment
    grad_W_s = np.dot(ds, h.T) / batch_size
    grad_b_s = np.sum(ds, axis=1, keepdims=True) / batch_size
   
    
    return grad_W1, grad_W2, grad_b1, grad_b2, grad_W_s, grad_b_s


In [151]:
def sentiment_prediction_model(W_s, b_s, h):
    # Logistic regression for sentiment classification
    z_s = np.dot(W_s, h) + b_s
    pred_s = sigmoid(z_s)
    print("pred_s")
    print(pred_s.shape)
    return pred_s

In [145]:
def gradient_descent1( N, V, num_iters):

   # W1, W2, b1, b2 ,W_s,b_s= initialize_model(N,V, random_seed=282)
    batch_size = 10
    iters = 0
    C = 1
  
    for x, y ,z in get_batches(batch_size): 
       
        print("y")
        print(type(y))
        print(y)
        print(y.shape)
        print("x")
        print(type(x))
        print(x)
        print(x.shape)
        print("z")
        print(type(z))
        print(z.shape)

        iters += 1 
        if iters == num_iters: 
            break
       
            
        

In [183]:
def gradient_descent(N, V, num_iters, alpha=0.03,beta=0.05):
    sentiment_classes = 2

    W1, W2, b1, b2 ,W_s,b_s= initialize_model(N,V,1,random_seed = 282)
    batch_size = N
    iters = 0
    C = 1
  
    for x, y ,y_sentiment in get_batches(batch_size): 
        
        y_sentiment = y_sentiment.reshape(1, -1)
      
        z, h  = forward_prop(x, W1, W2, b1, b2,W_s,b_s)
        pred_s= sentiment_prediction_model(W_s, b_s, h)
        

        
        yhat = softmax(z)
        
        cost = compute_cost(y, yhat, batch_size)
        sentiment_loss = compute_sentiment_cost(y_sentiment,pred_s, batch_size)  # y_sentiment are the true sentiment labels
        total_loss = beta * cost + (1 - beta) * sentiment_loss

        print(cost)
        if ( (iters+1) % 10 == 0):
            print(f"iters: {iters + 1} cost: {cost:.6f}")
        
        grad_W1, grad_W2, grad_b1, grad_b2,grad_W_s, grad_b_s = back_prop(x, yhat, y, h, W1, W2, b1, b2, W_s,b_s, pred_s,y_sentiment, batch_size)
        
        
        W1 -= alpha*grad_W1 
        W2 -= alpha*grad_W2
        b1 -= alpha*grad_b1
        b2 -= alpha*grad_b2

     # Update sentiment model parameters
        W_s -= alpha * grad_W_s
        b_s -= alpha * grad_b_s
        
        
        
        iters += 1 
        if iters == num_iters: 
            break
        if iters % 100 == 0:
            alpha *= 0.66
            
    return W1, W2, b1, b2

In [184]:
C = 1
N = 100
word2Ind, Ind2word = get_dict(data)
W1, W2, b1, b2 ,W_s,b_s= initialize_model(N,vocabsize, 2,random_seed = 282)

V = len(word2Ind)
num_iters = 5
W1, W2, b1, b2 = gradient_descent( N, vocabsize, num_iters)


pred_s
(1, 100)
13.692964676981592
pred_s
(1, 100)
9.029142731547003
pred_s
(1, 100)
4.579901300712
pred_s
(1, 100)
0.7622023533647546
pred_s
(1, 100)
0.22175012613268041


In [187]:
print(W1.shape)


(100, 1952)


In [186]:
print(W1)

[[0.36842629 0.39172236 0.12117024 ... 0.86283221 0.80354769 0.42350652]
 [0.51111431 0.72332409 0.93370004 ... 0.90829277 0.98011031 0.86157251]
 [0.87825143 0.04246476 0.58769782 ... 0.4974234  0.49062177 0.39280315]
 ...
 [0.64592789 0.7343668  0.62083031 ... 0.73095623 0.51407979 0.0657923 ]
 [0.63312484 0.06421244 0.79844273 ... 0.14403548 0.01158569 0.4114133 ]
 [0.63868317 0.13050883 0.07287024 ... 0.7325165  0.36827626 0.98729761]]


In [191]:
from matplotlib import pyplot
%config InlineBackend.figure_format = 'svg'
words = ['mud']

embs = (W1.T + W2)/2.0
 

idx = [word2Ind[word] for word in words]
X = embs[idx, :]
print(X.shape, idx) 

KeyError: 'mud'

In [ ]:
result= compute_pca(X, 2)
pyplot.scatter(result[:, 0], result[:, 1])
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

In [ ]:
result= compute_pca(X, 4)
pyplot.scatter(result[:, 3], result[:, 1])
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 3], result[i, 1]))
pyplot.show()

## Domain Adaptation

In [192]:
def load_word_embeddings(domain):
    # Load pre-trained word embeddings for domain
    # This should return a dictionary of word vectors
    pass

In [193]:
def load_corpus_frequency(domain):
    # Load or calculate word frequencies for the domain corpus
    # This should return a dictionary of word frequencies
    pass


In [194]:
# Sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [195]:
# Domain relevance function using Sørensen-Dice coefficient
def domain_relevance(w, freq_P, freq_Q):
    return 2 * freq_P[w] * freq_Q[w] / (freq_P[w] + freq_Q[w])

In [196]:
# Information transfer function
def information_transfer(phi_w, lambda_):
    return sigmoid(lambda_ * phi_w)

In [197]:
# Loss function
def compute_loss(W_p, W_q, L, freq_P, freq_Q, lambda_):
    loss_Q = 0  # Initialize domain Q loss
    for w in L:
        phi_w = domain_relevance(w, freq_P, freq_Q)
        t_w = information_transfer(phi_w, lambda_)
        loss_Q += t_w * np.linalg.norm(W_p[w] - W_q[w])**2
    return loss_Q

In [198]:
# Hyperparameters
learning_rate = 0.01
lambda_ = 0.1  # Hyperparameter for scaling the domain correlation function
epochs = 10    # Number of epochs to train

In [199]:
# Load data
W_s = load_word_embeddings('P')  # Pre-trained embeddings from domain P
freq_P = load_corpus_frequency('P')  # Word frequencies in domain P
freq_Q = load_corpus_frequency('Q')  # Word frequencies in domain Q

# Initialize domain Q embeddings
W_q = np.random.rand(*W_s.shape)

# Create word list L of words that appear in both domains
L = list(set(freq_P.keys()) & set(freq_Q.keys()))


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# Training loop
for epoch in range(epochs):
    # Initialize gradient accumulator
    grad_accumulator = np.zeros_like(W_q)

    # Compute the gradients for each word
    for word in L:
        idx = word_indices[word]
        phi_w = domain_relevance(word, freq_P, freq_Q)
        t_w = information_transfer(phi_w, lambda_)
        
        # Gradient of the loss with respect to the word vector
        grad_w = 2 * t_w * (W_q[idx] - W_p_array[idx])
        
        # Accumulate the gradients
        grad_accumulator[idx] += grad_w

    # Update W_q with the accumulated gradients
    W_q -= learning_rate * grad_accumulator

    # Calculate and print the loss (optional)
    loss_Q = compute_loss(W_p_array, W_q, L, freq_P, freq_Q, lambda_)
    print(f'Epoch {epoch}, Loss: {loss_Q}')
